<a href="https://colab.research.google.com/github/pedrohcaminha/ProjetoPLN/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 baixando os dados

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download hassanamin/atis-airlinetravelinformationsystem

atis-airlinetravelinformationsystem.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! unzip atis-airlinetravelinformationsystem.zip

Archive:  atis-airlinetravelinformationsystem.zip
replace atis_intents.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: atis_intents.csv        
  inflating: atis_intents_test.csv   
  inflating: atis_intents_train.csv  


In [7]:
! ls

atis-airlinetravelinformationsystem.zip  atis_intents_test.csv	 kaggle.json
atis_intents.csv			 atis_intents_train.csv  sample_data


# 2 lendo e separando os dados

importando bibliotecas

In [8]:
import spacy
import csv
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import spatial
import nltk
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers.embeddings import Embedding
from keras.layers import Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers import Input, Dense
from keras.models import Sequential

from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


dados de treino e teste

In [9]:
train = pd.read_csv("atis_intents_train.csv", header=None)
test = pd.read_csv("atis_intents_test.csv", header=None)

# 3 pre-processamento

stop words

In [10]:
from nltk.corpus import stopwords
words = set(stopwords.words("english"))
print(words)

{'off', 'it', 'your', "it's", 'they', 'having', 'there', 'each', 'before', 'own', 'shan', "weren't", "wasn't", 'ourselves', 'above', 'at', 'than', 'no', 'are', 'below', 'up', 'any', "you're", 'that', "you've", 'so', 'ain', 'shouldn', 'a', 'm', 'am', 'couldn', 'wasn', 'weren', 'has', 'just', 'mightn', 'don', 'hasn', 'you', 'her', 'needn', 'can', 'our', 'why', 'yourself', 'she', 'under', 'themselves', 'i', 'other', 'out', "didn't", 'how', "shouldn't", "should've", 'wouldn', 've', 'ma', 'does', 'about', "mightn't", 'and', 'them', 'been', "haven't", "you'd", 'once', 'during', 'or', 'into', 'nor', 'some', 'the', 'in', 'were', 'who', 'haven', 'we', "hadn't", 'him', 'an', 'down', 'myself', 'same', 'doing', 'too', 'while', 'my', 'its', 'but', 'with', 'which', 'do', 'what', "that'll", 'yours', 'these', 'their', 'doesn', "she's", 'again', 'then', "won't", 'such', 'himself', 'few', 'had', "aren't", 'is', 'from', 'more', 'aren', 'me', 'mustn', 'theirs', 'here', 'will', 'being', 'o', "wouldn't", 'o

In [11]:
train['text'] = train[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))
test['text'] = test[1].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))

train['text'] = train['text'].str.replace('\d+', '')
test['text'] = test['text'].str.replace('\d+', '')

In [12]:
text = train['text']
labels = train[0]
test_text = test['text']
test_labels = test[0]

In [13]:
train.head()

,0,1,text
0,atis_flight,i want to fly from boston at 838 am and arriv...,want fly boston arrive denver morning
1,atis_flight,what flights are available from pittsburgh to...,flights available pittsburgh baltimore thursda...
2,atis_flight_time,what is the arrival time in san francisco for...,arrival time san francisco flight leaving was...
3,atis_airfare,cheapest airfare from tacoma to orlando,cheapest airfare tacoma orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...,round trip fares pittsburgh philadelphia dollars


# 4 vetorizando palavras

In [14]:
from keras.preprocessing.text import Tokenizer
tok = Tokenizer()
tok.fit_on_texts(text)
word_index = tok.word_index

In [54]:
word_index['plane']

227

In [16]:
max_vocab_size = len(word_index) + 1
input_length = 25

In [17]:
train_data_tokens = tok.texts_to_sequences(text)
test_data_tokens = tok.texts_to_sequences(test_text)

In [18]:
train_input = pad_sequences(train_data_tokens, input_length)
test_input = pad_sequences(test_data_tokens, input_length)

# 5 codificando os rotulos

In [19]:
a = labels

label_transformer = preprocessing.LabelEncoder()
label_transformer.fit(labels)

LabelEncoder()

In [20]:
labels = label_transformer.transform(labels)
b = labels
test_labels = label_transformer.transform(test_labels)
c = labels

In [21]:
labels = to_categorical(np.asarray(labels))
d = labels
test_labels = to_categorical(np.asarray(test_labels))

In [22]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# 6 separação de treino e teste (para treinamento)

In [23]:
X_train, X_val, y_train, y_val = train_test_split(train_input, labels, test_size=0.2, random_state=42)
X_train

array([[  0,   0,   0, ...,  52, 477, 139],
       [  0,   0,   0, ..., 105,  50,  64],
       [  0,   0,   0, ...,   5,   7,   3],
       ...,
       [  0,   0,   0, ...,   2,  10,   9],
       [  0,   0,   0, ..., 218, 137,  93],
       [  0,   0,   0, ...,  42,  46,   3]], dtype=int32)

# 7 deep learning

In [36]:
model = Sequential()
model.add(Embedding(max_vocab_size, 300, input_length=input_length, trainable=False))
model.add(Conv1D(filters=32, kernel_size=8, activation='selu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='selu'))
model.add(Dense(8, activation='sigmoid'))

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 300)           189600    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 18, 32)            76832     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 9, 32)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2890      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 88        
Total params: 269,410
Trainable params: 79,810
Non-trainable params: 189,600
___________________________________________

In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=6, verbose=1)

Epoch 1/6
121/121 [==============================] - 2s 9ms/step - loss: 1.0786 - accuracy: 0.7262
Epoch 2/6
121/121 [==============================] - 1s 9ms/step - loss: 0.6141 - accuracy: 0.8318
Epoch 3/6
121/121 [==============================] - 1s 9ms/step - loss: 0.4242 - accuracy: 0.8947
Epoch 4/6
121/121 [==============================] - 1s 9ms/step - loss: 0.3237 - accuracy: 0.9138
Epoch 5/6
121/121 [==============================] - 1s 9ms/step - loss: 0.2614 - accuracy: 0.9297
Epoch 6/6
121/121 [==============================] - 1s 9ms/step - loss: 0.1732 - accuracy: 0.9591


In [39]:
model.evaluate(X_val, y_val)

31/31 [==============================] - 0s 4ms/step - loss: 0.2421 - accuracy: 0.9307


[0.2420656830072403, 0.9307135343551636]

In [40]:
predictions = model.predict(test_input)

In [41]:
def acc(y_true, y_pred):
    return np.equal(np.argmax(y_true, axis=-1), np.argmax(y_pred, axis=-1)).mean()

print(acc(test_labels, predictions))

0.94375


In [42]:
def get_intent(sentence):
  data = [[sentence]]
  df = pd.DataFrame(data)
  entrada = df[0].apply(lambda x: ' '.join([word for word in x.split() if word not in (words)]))
  entrada = tok.texts_to_sequences(entrada)
  entrada = pad_sequences(entrada, input_length)
  prediction = model.predict(entrada)
  # return np.argmin(prediction)
  return a[np.where(b == np.argmax(prediction))[0][0]]

#  8 intencoes

In [44]:
get_intent("what kinds of planes are used by american airlines")

'atis_aircraft'

In [45]:
get_intent("show me the first class fares from boston to denver")

'atis_airfare'

In [46]:
get_intent("show me the flights from boston to denver")

'atis_flight'

# 9 generating responses

In [75]:
intents = {}
for i in range(8):
  intents.update({a[np.where(b == i)[0][0]]: []})

intents['atis_abbreviation'].append("it means ____")
intents['atis_aircraft'].append("the aircraft used is ______")
intents['atis_airfare'].append("the price is ______")
intents['atis_airfare'].append("the price in _______ can vary from ______ to _____")
intents['atis_airfare'].append("the price in ______ is")
intents['atis_airfare'].append("the price is ______")
intents['atis_airline'].append("the airline is _____")
intents['atis_flight'].append("the available flights are _____")
intents['atis_flight_time'].append("the time is _____")
intents['atis_ground_service'].append("the service will be _____ available at ______")
intents['atis_quantity'].append("there are at least _______")

intents

{'atis_abbreviation': ['it means ____'],
 'atis_aircraft': ['the aircraft used is ______'],
 'atis_airfare': ['the price is ______',
  'the price in _______ can vary from ______ to _____',
  'the price in ______ is',
  'the price is ______'],
 'atis_airline': ['the airline is _____'],
 'atis_flight': ['the available flights are _____'],
 'atis_flight_time': ['the time is _____'],
 'atis_ground_service': ['the service will be _____ available at ______'],
 'atis_quantity': ['there are at least _______']}

In [92]:
! pip install termcolor
from termcolor import colored


In [99]:
import random
def response(text):
  intent = get_intent(text)
  print(colored(intent, 'red'))
  print(colored("BOT: " + random.choice(intents[intent]), 'green'))

In [100]:
entrada = input("YOU: ")
response(entrada)

YOU: which flights are available from denver to new mexico
atis_flight
BOT: the available flights are _____
